In [1]:
import numpy as np
from scipy.stats import binom
from scipy import optimize

In [2]:
from scipy.stats import binom

def B_zk(z, k, p):
    """Computes B_z,k(p) as the CDF of a binomial distribution."""
    return binom.cdf(k, z, p)

def h_z(k, N, lambd, z):
    """Computes h_z(k, N, λ) based on the piecewise function using B_z,k(1 - λ)."""
    B_zk_val = B_zk(z, k, 1 - lambd)
    if 0 <= z <= k:
        return 1
    elif k + 1 <= z <= N + 1:
        B_z_minus_1_k_val = B_zk(z - 1, k, 1 - lambd)
        return ((N - z + 1) * B_zk_val + z * B_z_minus_1_k_val) / (N + 1)
    else:
        raise ValueError("z is out of range")

def g_z(k, N, lambd, z):
    """Computes g_z(k, N, λ) based on the piecewise function using B_z,k(1 - λ)."""
    B_zk_val = B_zk(z, k, 1 - lambd)
    if 0 <= z <= k:
        return (N - z + 1) / (N + 1)
    elif k + 1 <= z <= N + 1:
        return B_zk_val * (N - z + 1) / (N + 1)
    else:
        raise ValueError("z is out of range")

def kappa_z(k, N, delta, lambd, z):
    """Computes κ_z(k, N, δ) based on h_z and h_{z+1}."""
    h_z_val = h_z(k, N, lambd, z)
    h_z_plus_1_val = h_z(k, N, lambd, z + 1)
    return (delta - h_z_plus_1_val) / (h_z_val - h_z_plus_1_val)

def zeta_lambda(k, N, delta, z, lambd):
    """Computes ζ̅_λ(k, N, z, δ) using κ_z and g_z."""
    kappa = kappa_z(k, N, delta, lambd, z)
    g_z_val = g_z(k, N, lambd, z)
    g_z_plus_1_val = g_z(k, N, lambd, z + 1)
    return (1 - kappa) * g_z_plus_1_val + kappa * g_z_val

def find_z_hat(k, N, lambd, delta):
    """Finds the largest integer z such that h_z(k, N, λ, z) ≥ δ."""
    z_hat = k
    for z in range(N + 1):
        if h_z(k, N, lambd, z) >= delta:
            z_hat = z
    if z_hat == -1:
        raise ValueError("No z found satisfying h_z(k, N, λ, z) >= δ.")
    return z_hat

def epsilon_lambda(k, N, delta, lambd):
    """Computes ε̅(k, N, δ) using B_z,k(1 - λ)."""
    B_Nk = B_zk(N, k, 1 - lambd)
    if delta <= B_Nk:
        return 1
    else:
        z_hat = find_z_hat(k, N, lambd, delta)
        zeta = zeta_lambda(k, N, delta, z_hat, lambd)
        return 1 - zeta / delta

def minimum_test_number(epsilon, delta, lambd, r):
    """
    Implements Algorithm 1 for robust verification with B_z,k(1 - λ).
    
    Args:
        epsilon: ϵ, target bound in the range (0, 1).
        delta: δ, error tolerance in the range (0, 1).
        lambd: λ, parameter in the range (0, 1).
        r: parameter in the range [0, 1].
    
    Returns:
        (k_min, N_min): Tuple of integers representing the minimum k and N.
    """
    if r == 0:
        k_min = 0
    else:
        for k in range(int(1e6)):
            M = k
            while True:
                B_Mk = B_zk(M, k, (1 - lambd) * r * epsilon)
                if B_Mk >= 1 - delta:
                    if M >= k + 1 and epsilon_lambda(k, M, delta, lambd) <= epsilon:
                        break
                    M += 1
                else:
                    break
            if M >= k + 1 and epsilon_lambda(k, M, delta, lambd) <= epsilon:
                k_min = k
                break

    N_min = None
    for N in range(k_min + 1, int(1e6)):
        if epsilon_lambda(k_min, N, delta, lambd) <= epsilon:
            N_min = N
            break

    return k_min, N_min

In [ ]:
# Example parameters
epsilon = 0.2
delta = 0.01
lambd = 0.5  # λ
r = 0.5

k_min, N_min = minimum_test_number(epsilon, delta, lambd, r)
print(f"Minimum k: {k_min}, Minimum N: {N_min}")

In [ ]:
def B(k, p):
    return lambda M : binom.cdf(k, M, p)

def epsilon_l_max(k, M, delta):
    return s+1/(v*lambda)*np.sqrt((s*np.log(1/delta))/M)+np.log(1/delta)/(2*v**2*lambda*M)+2/(lambda*N)


def verification_algorithm(lambda, epsilon, delta, r):
    if r = 0:
        k_min = 0
    else:
        M = 0
        while (M < k + 1) & (epsilon_l() > epsilon):
            k = 0
            p = (1-lambda)*r*epsilon
            M = optimize.fsolve(B(k, p), )
        
        
            
    return N_min, k_min

In [3]:
def N_min_analytic(delta, epsilon):
    return 144*(np.log(1/delta+0.5))/epsilon

def N_min_numeric(delta, epsilon):
    return 67*np.log(1/delta)/epsilon

In [6]:
N_min_analytic(0.01, 0.06)

11064.378545997914

In [7]:
N_min_numeric(0.01, 0.06)

5142.440041020036

In [28]:
total_data = (3000*20)
time_per_state = 1.5
time_per_sensing_round = total_data*time_per_state/3600
print(f"Time for each data point: {time_per_sensing_round} hours")

Time for each data point: 25.0 hours


In [ ]:
M = 3000
f = 0.03 # increasing f will increase the confidence level. does this make sense?
k = f*M
r = 1/2 # increasing robustness will decrease the confidence level... does this make sense?
epsilon = 0.1
p = 1/2*r*epsilon
confidence_level = binom.cdf(k, M, p)
print(confidence_level)

0.962002418242284
